In [1]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import json
from datetime import datetime, timedelta

pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

# if not mt5.initialize():
#     print("initialize() failed")
# else:
#     print("MT5 successfully initialised.\n")

In [2]:
current_dir = os.getcwd()
#current_path = os.path.abspath(__file__)
#current_dir = os.path.dirname(current_path)
config_file_name = 'config.json'
config_file_path = os.path.join(current_dir, config_file_name)
with open(config_file_path, 'r') as config_file:
        config = json.load(config_file)

symbols = config['symbols']
api_token = config['api_token']
mapping_file = config['mapping_file_path']
sleep_time = config['sleep_time']
account=config['execute_account']
password=config['password']
server=config['server']
Port_TPSL = config['Port_TPSL']
starting_cash = config['starting_cash']

In [3]:
authorized=mt5.login(account, password=password, server=server)
if authorized:
    print(mt5.account_info())
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


AccountInfo(login=2117, trade_mode=0, leverage=1, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=1, currency_digits=2, fifo_close=False, balance=999972453.75, credit=0.0, profit=0.0, equity=999972453.75, margin=0.0, margin_free=999972453.75, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jeff Ng', server='KIBB-Prod', currency='MYR', company='Kenanga Investment Bank Berhad')


In [4]:
mt5_symbol = sm.get_ticker_list(symbols, file_path=mapping_file)
symbols = [symbol + "SE" for symbol in symbols]
price_df = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(price_df)
price_df.columns = mt5_symbol
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
print(f'latest_weights: {latest_weights}')
#ADD DUMMY WEIGHTS HERE TO TEST
total_investment = starting_cash
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')

latest_weights: RHBBANK    0.593484
MAYBANK    0.253123
YNHPROP    0.153393
Name: 2023-12-01, dtype: float64


In [6]:
if previous_allocation_df is not None and previous_portfolio_df is not None:
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)
for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.47,10800.0,0.593484,0.0,10800.0,0.0
1,MAYBANK,9.00,2800.0,0.253123,0.0,2800.0,0.0
2,YNHPROP,4.97,3000.0,0.153393,0.0,3000.0,0.0


In [14]:
import time
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')

Placing entry order for RHBBANK
Order executed successfully, transaction ID = 586555
Placing entry order for MAYBANK
Order executed successfully, transaction ID = 586556
Placing entry order for YNHPROP
Order executed successfully, transaction ID = 586557
Order with ticket 586555 not found or error occurred, error code = (1, 'Success')
Order with ticket 586556 not found or error occurred, error code = (1, 'Success')
Order with ticket 586557 not found or error occurred, error code = (1, 'Success')
orders_list: [None, None, None]
deleted_volumes: []


In [18]:
def get_opening_orders(stock_symbols, weights):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        return None, pd.DataFrame()  # Return an empty DataFrame if MT5 fails to initialize

    all_positions = []
    open_prices = {}
    for symbol in stock_symbols:
        positions = mt5.positions_get(symbol=symbol)
        today_open_price = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_D1, 0, 1)
        if positions is None:
            print(f"No positions for {symbol}, error code={mt5.last_error()}")
        elif len(positions) > 0:
            all_positions.extend(list(positions))
            open_prices[symbol] = today_open_price[0]['open']
    df = pd.DataFrame()
    if all_positions:
        df = pd.DataFrame(all_positions, columns=all_positions[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df = df.drop(['time_msc', 'time_update', 'time_update_msc', 'identifier', 'reason', 
                      'sl', 'tp', 'swap', 'comment', 'external_id'], axis=1)
        df['position_change'] = (df['price_current'] / df['price_open']) - 1
        df['value'] = df['volume'] * df['price_open']
        df['weights'] = df['symbol'].map(weights)
        df['portfolio_change_position'] = df['weights'] * df['position_change']
    total_portfolio_change_position = df['portfolio_change_position'].sum() if not df.empty else 0
    print(f"Total portfolio change position: {total_portfolio_change_position:.4%}")
    return total_portfolio_change_position, df
mt5_symbol = ['RHBBANK','MAYBANK','YNHPROP']
port_change, opening_df = get_opening_orders(mt5_symbol, latest_weights)
opening_df


Total portfolio change position: -0.2787%


,ticket,time,type,magic,volume,price_open,price_current,profit,symbol,position_change,value,weights,portfolio_change_position
0,586555,2023-12-04 17:07:11,0,1116,10800.0,5.47,5.45,-216.0,RHBBANK,-0.003656,59076.0,0.593484,-0.002170
1,586556,2023-12-04 17:07:11,0,1116,2800.0,9.00,9.00,0.0,MAYBANK,0.000000,25200.0,0.253123,0.000000
2,586557,2023-12-04 17:07:11,0,1116,3000.0,4.97,4.95,-60.0,YNHPROP,-0.004024,14910.0,0.153393,-0.000617


In [29]:
from datetime import datetime, timedelta

def get_filled_orders(symbol_list):
    from_date = datetime.now()
    to_date = datetime.now() + timedelta(days=1)

    print(f"from_date: {from_date}")
    print(f"to_date: {to_date}")
    all_deals = []
    for symbol in symbol_list:
        deals = mt5.history_orders_get(from_date, to_date)
        if deals is None:
            print(f"No deals for {symbol}, error code={mt5.last_error()}")
        elif len(deals) > 0:
            all_deals.extend(list(deals))
    if not all_deals:
        print("No deals found for any of the specified symbols.")
        return pd.DataFrame()
    df = pd.DataFrame(all_deals, columns=all_deals[0]._asdict().keys())
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
    # Drop columns
    df = df.drop(['time_setup', 'time_done_msc', 'time_setup_msc', 'time_expiration', 'sl',
                  'type_time', 'type_filling', 'volume_current', 'reason', 'price_open', 'tp',
                  'position_id', 'position_by_id', 'external_id', 'price_stoplimit'], axis=1)
    df = df.drop_duplicates()
    filled_orders = df[df['state'] == 4]
    filled_orders = filled_orders[filled_orders['symbol'].isin(symbol_list)]
    return filled_orders

filled_orders = get_filled_orders(mt5_symbol)
filled_orders
################
current_time = datetime.now()
# Filter for orders done in the last 5 minutes
five_minutes_ago = current_time - timedelta(minutes=15)
recent_filled_orders = filled_orders[pd.to_datetime(filled_orders['time_done']) >= five_minutes_ago]
recent_filled_orders

from_date: 2023-12-04 17:12:49.069106
to_date: 2023-12-05 17:12:49.069106


,ticket,time_done,type,state,magic,volume_initial,price_current,symbol,comment
260,586555,2023-12-04 17:07:11,2,4,1116,10800.0,5.45,RHBBANK,Handpick Entry
261,586556,2023-12-04 17:07:11,2,4,1116,2800.0,9.00,MAYBANK,Handpick Entry
262,586557,2023-12-04 17:07:11,2,4,1116,3000.0,4.95,YNHPROP,Handpick Entry


In [30]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.47,10800.0,0.593484,0.0,10800.0,0.0
1,MAYBANK,9.00,2800.0,0.253123,0.0,2800.0,0.0
2,YNHPROP,4.97,3000.0,0.153393,0.0,3000.0,0.0


In [ ]:
def close_all_positions(df, comment):
    for _, row in df.iterrows():
        symbol = row['symbol']
        volume = row['volume']
        # Fetch the current bid price for the symbol
        symbol_info = mt5.symbol_info_tick(symbol)
        if symbol_info is None:
            print(f"Failed to get symbol info for {symbol}, error code =", mt5.last_error())
            continue
        current_bid = symbol_info.bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": mt5.ORDER_TYPE_SELL,
            "price": current_bid,
            "deviation": 20,  # Deviation in points
            "magic": 20231207,       # Magic number, if needed
            "comment": comment,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Failed to close position for {symbol}, error code: {result.retcode}")
        else:
            print(f"Position on {symbol} closed successfully.")

close_all_positions(opening_df, "Closing all positions")

In [ ]:
allocation_df['Entry Units']

In [ ]:
if not opening_df.empty:
    print("Opening orders:")
    print(opening_df)
else:
    print("No opening orders found.")

In [ ]:
if opening_df is not None and filled_orders is not None:
    #drop Entry Units and Exit Units
    allocation_df = allocation_df.drop(['Share Price','Holding Units',
                                        'Entry Units','Exit Units'], axis=1)
    allocation_df['Entry Price'] = opening_df['price_open']
    allocation_df['Current Price'] = opening_df['price_current']
    entry_orders = filled_orders[filled_orders['type'] == 0]
    exits_orders = filled_orders[filled_orders['type'] == 1]

    # Step 2: Group by 'symbol' and sum the 'volume_initial'
    opening_units = opening_df.groupby('symbol')['volume'].sum().reset_index()
    opening_units.rename(columns={'volume': 'Holding Units', 'symbol': 'Share Symbol'}, inplace=True)
    entry_units = entry_orders.groupby('symbol')['volume_initial'].sum().reset_index()
    entry_units.rename(columns={'volume_initial': 'Entry Units', 'symbol': 'Share Symbol'}, inplace=True)
    exit_units = exits_orders.groupby('symbol')['volume_initial'].sum().reset_index()
    exit_units.rename(columns={'volume_initial': 'Exit Units', 'symbol': 'Share Symbol'}, inplace=True)
    exit_prices = exits_orders.groupby('symbol')['price_current'].mean().reset_index()
    exit_prices.rename(columns={'price_current': 'Exit Price', 'symbol': 'Share Symbol'}, inplace=True)

    # Step 3: Merge with allocation_df
    allocation_df = pd.merge(allocation_df, exit_prices, on='Share Symbol', how='left')
    allocation_df = pd.merge(allocation_df, opening_units, on='Share Symbol', how='left')
    allocation_df = pd.merge(allocation_df, entry_units, on='Share Symbol', how='left')
    allocation_df = pd.merge(allocation_df, exit_units, on='Share Symbol', how='left')
    allocation_df['Exit Price'].fillna(0, inplace=True)
    allocation_df['Holding Units'].fillna(0, inplace=True)
    allocation_df['Entry Units'].fillna(0, inplace=True)
    allocation_df['Exit Units'].fillna(0, inplace=True)
    allocation_df['Entry Price'].fillna(0, inplace=True)
    allocation_df['Current Price'].fillna(0, inplace=True)
else:
    print("No opening_df or filled_orders found.")

In [ ]:
import pandas as pd
from datetime import datetime
import os

def generate_daily_log(allocation_df, initial_capital=10000, log_folder="daily_portfolio", log_file_name="balance.csv"):
    log_file_path = os.path.join(log_folder, log_file_name)
    if os.path.exists(log_folder) and os.path.isfile(log_file_path):
        previous_log = pd.read_csv(log_file_path)
        cash_balance = previous_log['Cash Balance'].iloc[-1]
        portfolio_value = (allocation_df['Holding Units'] * allocation_df['Current Price']).sum()
        initial_cap = cash_balance + portfolio_value
    else:
        initial_cap = initial_capital
    purchase_cost = (allocation_df['Entry Units'] * allocation_df['Entry Price']).sum()
    sale_proceed = (allocation_df['Exit Units'] * allocation_df['Exit Price']).sum()
    cash_balance = initial_cap - purchase_cost + sale_proceed
    portfolio_value = (allocation_df['Holding Units'] * allocation_df['Current Price']).sum()
    total_value = cash_balance + portfolio_value

    log_entry = pd.DataFrame({
        'Date': [datetime.now().strftime('%Y-%m-%d')],
        'Initial Capital': [initial_cap],
        'Purchase Cost': [purchase_cost],
        'Sale Proceed': [sale_proceed],
        'Cash Balance': [cash_balance],
        'Portfolio Value': [portfolio_value],
        'Total Value': [total_value]
    })
    return log_entry

log_entry = generate_daily_log(allocation_df)
mt2.save_df_to_csv(log_entry, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)